In [2]:
import torch
import numpy as np

In [3]:
from data_utils.step_full_dataset import Step_2D_dataset_for_GNOT, StepDataset

In [4]:
data_path = r'C:\Users\Noahc\Documents\USYD\tutorial\python_utils\backward_facing_step_normalized.npy'

In [83]:
# from data_utils.utils import UnitTransformer
# def create_a_normalizer(un_normalized_data):

#         # Flatten
#         n_channels = un_normalized_data.shape[-1]
#         batches_and_nodes = torch.prod(torch.tensor(un_normalized_data.shape[:-1])).item()
#         all_features = un_normalized_data.reshape(batches_and_nodes,n_channels)
        
#         return UnitTransformer(all_features)

# class Step_2D_dataset_for_GNOT():
#     def __init__(self, 
#                 data_path,
#                 normalize_y = False, 
#                 normalize_x = False,
#                 normalize_f = False,
#                 boundary_input_f = False,
#                 subsample_ratio = 1.0):
    
#         print('\nCalculating Step Dataset Normalizers')
#         # Normalizer settings:
#         self.normalize_y = normalize_y
#         self.normalize_x = normalize_x
#         self.normalize_f = normalize_f

#         self.data_dict   = np.load(data_path,allow_pickle=True).item()
#         self.Re = torch.tensor(self.data_dict['Re'])
#         self.x = torch.tensor(self.data_dict['Points'])
#         self.y = torch.tensor(self.data_dict['Solutions'])
#         self.n_internal = self.data_dict['Points'].shape[0]
#         self.n_cases = self.data_dict['Solutions'].shape[0]
#         self.bc = None

#         print('Dataset loaded has the following keys: ', self.data_dict.keys())
#         print(f'Reynolds Number Range: {self.Re.min()}-{self.Re.max()}')
#         print(f'Total Cases: {self.n_cases}')
#         print(f'Total Internal Cells: {self.n_internal} (before any subsampling)')
#         print(f'Patch Names: {self.data_dict["Boundary"].keys()}')

#         # Store the indices of all patches
#         self.patch_indices = dict.fromkeys(self.data_dict["Boundary"])
#         starting_bc_index = self.x.shape[0]

#         for patch in self.data_dict["Boundary"]:
#             num_points = self.data_dict["Boundary Points"][patch].shape[0]
#             self.x = torch.concat([self.x, torch.tensor(self.data_dict["Boundary Points"][patch])], dim=0)
#             self.y = torch.concat([self.y, torch.tensor(self.data_dict["Boundary"][patch])], dim=1)
#             self.patch_indices[patch] = np.arange(starting_bc_index,starting_bc_index+num_points)
#             starting_bc_index += num_points

#         if boundary_input_f: 
#             self.bc = {'Coords' : self.x[self.n_internal:,...],
#                        'Values' : self.y[:,self.n_internal:,...]
#                        }

#         print(f'Total Boundary Cells: {self.x.shape[0]-self.data_dict["Points"].shape[0]}\n')

#         # Get Normalizers
#         self.x_normalizer = create_a_normalizer(self.x)
#         self.y_normalizer = create_a_normalizer(self.y)
#         self.xi_normalizer = UnitTransformer(self.Re)

#         # Normalize if required
#         if self.normalize_x:
#             self.queries = self.x_normalizer.transform(self.x, inverse=False)
#             print(f'    Queries Normalized with Means: {self.x_normalizer.mean} and Stds: {self.x_normalizer.std}')
#             if boundary_input_f:
#                 self.bc['Coords'] = self.x_normalizer.transform(self.bc['Coords'], inverse=False)
#                 print(f'    BC coords Keys Normalized with Means: {self.x_normalizer.mean} and Stds: {self.x_normalizer.std}')

#         if self.normalize_y:
#             self.data_out = self.y_normalizer.transform(self.y, inverse=False)
#             print(f'    Solutions Normalized with Means: {self.y_normalizer.mean} and Stds: {self.y_normalizer.std}')
#             if boundary_input_f:
#                 self.bc['Values'] = self.y_normalizer.transform(self.bc['Values'], inverse=False)
#                 print(f'    BC values Keys Normalized with Means: {self.y_normalizer.mean} and Stds: {self.y_normalizer.std}')
    
#         if self.normalize_f:
#             self.Re = self.xi_normalizer.transform(self.Re, inverse=False)
#             print(f'    Re Keys Normalized with Means: {self.xi_normalizer.mean} and Stds: {self.xi_normalizer.std}')
            
#         # Only Subsample the internal field (and after normalizer are calculated)
#         if subsample_ratio != 1.0:
#             # Interior random samples
#             sample = list(np.random.default_rng().choice(self.n_internal, size=int(self.n_internal*subsample_ratio), replace=False))
#             # All boundary points
#             sample += list(np.arange(self.n_internal,self.x.shape[0]))
#             # points reduced
#             n_points_reduced = self.n_internal - int(self.n_internal*subsample_ratio)

#             self.x = self.x[sample,...]
#             self.y = self.y[:,sample,...]

#             for patch in self.patch_indices:
#                 self.patch_indices[patch] = self.patch_indices[patch] - n_points_reduced

#             print(f'\nTotal Internal Cells reduced by {n_points_reduced} to: {self.x.shape[0]} (after subsampling)')

#         self.__update_dataset_config()

#     def __update_dataset_config(self): 
#         self.config = {
#             'input_dim': self.x.shape[-1],
#             'theta_dim': 0,
#             'output_dim': self.y.shape[-1],
#             'branch_sizes': [2 if self.bc is not None else 1]
#         }


In [8]:
import plotly.graph_objects as go
import plotly

case_n = 0
channel = 2
subsample_perc = 0.3

dataset = Step_2D_dataset_for_GNOT(data_path=data_path, 
                                   subsample_ratio=subsample_perc, 
                                   normalize_f=True, 
                                   boundary_input_f=True)

fig = go.Figure()
fig.add_trace(go.Scatter(x=dataset.x[:,0],
                                y=dataset.x[:,1],
                                mode='markers',
                                marker=dict(
                                    color=dataset.y[case_n,:,channel],
                                    colorscale='Viridis',
                                    coloraxis='coloraxis'
                                    ),
                                text=dataset.y[case_n,:,channel],
                                name='Internal Field')) # hover text goes here


Calculating Step Dataset Normalizers
Dataset loaded has the following keys:  dict_keys(['Solutions', 'Re', 'Points', 'Boundary', 'Boundary Points'])
Reynolds Number Range: 26.0-225.0
Total Cases: 200
Total Internal Cells: 12225 (before any subsampling)
Patch Names: dict_keys(['upperWall', 'inlet', 'outlet', 'lowerWall'])
Total Boundary Cells: 564

    Re Keys Normalized with Means: tensor([125.5000]) and Stds: tensor([57.8792])

Total Internal Cells reduced by 8558 to: 4231 (after subsampling)


Now time for

## Dataloader

In [85]:
class MultipleTensors():
    def __init__(self, x):
        self.x = x

    def to(self, device):
        self.x = [x_.to(device) for x_ in self.x]
        return self

    def __len__(self):
        return len(self.x)


    def __getitem__(self, item):
        return self.x[item]

In [86]:
class StepDataset():
    def __init__(self,dataset, train=True, inference=True, train_ratio=0.7, seed=42, random_coords = True):

        print('\nCreating Dataloader:')
        self.x = dataset.x
        self.Re = dataset.Re
        self.y = dataset.y
        self.patch_indices = dataset.patch_indices.copy()
        if dataset.bc is None:
            self.bc_values = None
            self.bc_coords = None
        else:
            self.bc_values = dataset.bc['Values'].clone()
            self.bc_coords = dataset.bc['Coords']

        self.random_coords = random_coords
        self.train = train
        self.data_splitter(train,inference,train_ratio,seed)

    def data_splitter(self, train, inference, train_ratio, seed):
        n_batches = self.y.shape[0]
        train_size = int(train_ratio * n_batches)
        test_size = n_batches - train_size
        if inference:
            seed_generator = torch.Generator().manual_seed(seed)

            # we want to pin the end points as training (to ensure complete inference)
            train_split,  test_split        = torch.utils.data.random_split(self.y[1:-1,...], [train_size-2, test_size], generator=seed_generator)
            test_split.indices = list(1 + np.array(test_split.indices))
            train_split.indices = list(1 + np.array(train_split.indices))
            train_split.indices.append(0)
            train_split.indices.append(-1)
        
            # The torch.utils.data.random_split() only gives objects with the whole datset or a integers, so we need to override these variables with the indexed datset split
            train_dataset,  test_dataset    = self.y[train_split.indices,...], self.y[test_split.indices,...]
            train_Re,    test_Re            = self.Re[train_split.indices], self.Re[test_split.indices]
            if dataset.bc is not None:
                train_bc, test_bc           = self.bc_values[train_split.indices,...], self.bc_values[test_split.indices,...]

            print(f'    Dataset Split up for inference using torch generator seed: {seed_generator.initial_seed()}')
        
        else:
            train_dataset,  test_dataset    = self.y[:train_size,...],  self.y[train_size:,...]
            train_Re,    test_Re            = self.Re[:train_size,...],     self.Re[train_size:,...]
            if dataset.bc is not None:
                train_bc, test_bc           = self.bc_values[:train_size,...], self.bc_values[train_size:,...]

            print(f'    Dataset Split up for High reynolds number extrapolation')

        if train:
            self.y = train_dataset
            self.Re = train_Re
            if dataset.bc is not None:
                self.bc_values = train_bc
            print(f'    Training Dataset Selected')
        else:
            self.y = test_dataset
            self.Re = test_Re
            if self.bc is not None:
                self.bc_values = test_bc
            print(f'    Testing Dataset Selected')

    def __len__(self):
        return len(self.Re)
    
    def __getitem__(self, idx):
        
        # randomize input coordinates
        if self.random_coords:
            indices = torch.randperm(self.x.shape[0])
            reverse_indices = torch.argsort(indices)
            reverse_patch_indices = dict.fromkeys(self.patch_indices)
            for patch in self.patch_indices:
                reverse_patch_indices[patch] = reverse_indices[self.patch_indices[patch]]
        else:
            indices = torch.arange(self.x.shape[0])
            reverse_indices = indices
            reverse_patch_indices = self.patch_indices.copy()
        
        in_keys     = self.Re[idx].float().reshape(1,1)
        in_bc_value = self.bc_values[idx,:,...].float()
        in_queries  = self.x[indices,...].float()
        out_truth   = self.y[idx,indices,...].float()
        
        if dataset.bc is None:
            input_f = in_keys
        else:
            input_f = in_keys
            input_f = [in_keys, in_bc_value, self.bc_coords.float()]

        index_list = {'All Indices' : reverse_indices,
                      'Boundary Indices' :reverse_patch_indices
                      }
        
        return in_queries, input_f, out_truth, index_list
    
dataloader = StepDataset(dataset, inference=False)


Creating Dataloader:
    Dataset Split up for High reynolds number extrapolation
    Training Dataset Selected


Now we need a way to locate the boundaries under randomized conditions

In [87]:
data_iter = iter(dataloader)

In [88]:
batch=next(data_iter)

x, xi, y, index = batch
print(y.shape)
print(y[:,2].mean(), y[:,1].mean(), y[:,0].mean())

fig = go.Figure()

fig.add_trace(go.Scatter(x=x[:,0],
                            y=x[:,1],
                            mode='markers',
                            marker=dict(
                                color=y[:,channel],
                                colorscale='Viridis',
                                coloraxis='coloraxis'
                                ),
                            text=y[:,channel],
                            name='All Field')) # hover text goes here

for patch in index['Boundary Indices']:
    fig.add_trace(go.Scatter(x=x[index['Boundary Indices'][patch],0],
                            y=x[index['Boundary Indices'][patch],1],
                            mode='markers',
                            marker=dict(
                                color=y[index['Boundary Indices'][patch],channel],
                                colorscale='Viridis',
                                coloraxis='coloraxis'
                                ),
                            text=y[index['Boundary Indices'][patch],channel],
                            name=patch)) # hover text goes here
fig.show()

torch.Size([3009, 3])
tensor(0.7809) tensor(-0.0349) tensor(0.4375)


In [89]:
x.shape

torch.Size([3009, 2])

In [90]:
y.shape

torch.Size([3009, 3])

In [91]:
for patch in index['Boundary Indices']:
    print(index['Boundary Indices'][patch].shape)

torch.Size([224])
torch.Size([31])
torch.Size([58])
torch.Size([251])


In [102]:

from torch.utils.data import DataLoader
data_loader = DataLoader(
                    dataset=dataloader,
                    batch_size=4,
                    shuffle=True
                )

for batch in data_loader: break
x, xi, y, index = batch

fig = go.Figure()

fig.add_trace(go.Scatter(x=x[0,:,0],
                            y=x[0,:,1],
                            mode='markers',
                            marker=dict(
                                color=y[0,:,channel],
                                colorscale='Viridis',
                                coloraxis='coloraxis'
                                ),
                            text=y[0,:,channel],
                            name='All Field')) # hover text goes here

for patch in index['Boundary Indices']:
    fig.add_trace(go.Scatter(x=x[0,index['Boundary Indices'][patch][0,:],0],
                            y=x[0,index['Boundary Indices'][patch][0,:],1],
                            mode='markers',
                            marker=dict(
                                color=y[0,index['Boundary Indices'][patch][0,:],channel],
                                colorscale='Viridis',
                                coloraxis='coloraxis'
                                ),
                            text=y[0,index['Boundary Indices'][patch][0,:],channel],
                            name=patch)) # hover text goes here
fig.show()

## Cavity Dataset

In [4]:
from data_utils.dataset_prep import create_loader, prepare_dataset
from train_utils.default_args import get_default_args

dataset_args, model_args, training_args = get_default_args()

# Override for Step Case
#dataset_args['name'] = 'Step'
dataset_args['file_path'] = r'C:\Users\Noahc\Documents\USYD\tutorial\python_utils\backward_facing_step_normalized.npy'
dataset_args['file_path'] = r'C:\Users\Noahc\Documents\USYD\tutorial\python_utils\cavity_with_bc_normalized.npy'
dataset_args['sub_x'] = 0.1

# 1. Dataset Preperations:
dataset = prepare_dataset(dataset_args, unsupervised=False)


Calculating Step Dataset Normalizers
Dataset loaded has the following keys:  dict_keys(['Solutions', 'Re', 'Points', 'Boundary', 'Boundary Points'])
Reynolds Number Range: 5.0-1000.0
Total Cases: 200
Total Internal Cells: 65536 (before any subsampling)
Patch Names: dict_keys(['movingWall', 'fixedWalls'])
Total Boundary Cells: 0

    Queries Normalized with Means: tensor([[0.5000, 0.5000]], dtype=torch.float64) and Stds: tensor([[0.2887, 0.2887]], dtype=torch.float64)
    Solutions Normalized with Means: tensor([[ 4.8097e-06,  6.7330e-07, -3.4039e-02]], dtype=torch.float64) and Stds: tensor([[0.2151, 0.1867, 0.1582]], dtype=torch.float64)
    Re Keys Normalized with Means: tensor([502.5000]) and Stds: tensor([289.3959])

Total Internal Cells reduced by 58983 to: 6553 (after subsampling)


In [5]:
dataset.data_dict["Boundary"]

{'movingWall': array([], shape=(200, 0, 3), dtype=float64),
 'fixedWalls': array([], shape=(200, 0, 3), dtype=float64)}

In [8]:
import numpy as np
raw = np.load(dataset_args['file_path'],allow_pickle=True).item()

In [10]:
raw.keys()

dict_keys(['Solutions', 'Re', 'Points', 'Boundary', 'Boundary Points'])

In [11]:
raw['Boundary']

{'movingWall': array([], shape=(200, 0, 3), dtype=float64),
 'fixedWalls': array([], shape=(200, 0, 3), dtype=float64)}

In [12]:
raw['Boundary Points']

{'movingWall': array([], shape=(0, 2), dtype=float64),
 'fixedWalls': array([], shape=(0, 2), dtype=float64)}

## L2 Validation Dataloader

In [21]:
output = np.load(r'Z:\PRJ-MLFluids\Retiro_results\test_step_v1\test_aMonitor_b0_c1_d0.2_h200_l2_loss.npy', allow_pickle=True).item()

In [22]:
output.keys()

dict_keys(['Model Configuration', 'Training Configuration', 'Data Configuration', 'Re Number', 'L2 Rel Loss', 'PDE 1 Abs Loss', 'PDE 2 Abs Loss', 'PDE 3 Abs Loss', 'BC D Abs Loss', 'BC VN Abs Loss', 'Inference Time', 'Label'])

In [24]:
len(output['Re Number'])

200

## Create Charts for Presentation

In [10]:
from torch.utils.data import DataLoader
from data_utils.step_full_dataset import Step_2D_dataset_for_GNOT, StepDataset
import plotly.graph_objects as go
import plotly
import numpy as np

data_path = r'C:\Users\Noahc\Documents\USYD\tutorial\python_utils\backward_facing_step_normalized.npy'
data_path = r'C:\Users\Noahc\Documents\USYD\tutorial\python_utils\cavity_with_bc_normalized.npy'
case_n = 0
channel = 0
subsample_perc = 1.0

dataset = Step_2D_dataset_for_GNOT(data_path=data_path, 
                                   subsample_ratio=subsample_perc, 
                                   normalize_f=True, 
                                   boundary_input_f=True)


Calculating Step Dataset Normalizers
Dataset loaded has the following keys:  dict_keys(['Solutions', 'Re', 'Points', 'Boundary', 'Boundary Points'])
Reynolds Number Range: 5.0-1000.0
Total Cases: 200
Total Internal Cells: 65536 (before any subsampling)
Patch Names: dict_keys(['movingWall', 'fixedWalls'])
Total Boundary Cells: 1026

    Re Keys Normalized with Means: tensor([502.5000]) and Stds: tensor([289.3959])


In [20]:
index = np.random.randint(low=0,high=65536,size=1000,dtype=int)

fig = go.Figure()
fig.add_trace(go.Scatter(x=dataset.x[index,0],
                                y=dataset.x[index,1],
                                mode='markers',
                                marker=dict(
                                    color=dataset.y[case_n,index,channel],
                                    colorscale='Viridis',
                                    coloraxis='coloraxis'
                                    ),
                                text=dataset.y[case_n,index,channel],
                                name='Internal Field')) # hover text goes here

for patch in dataset.patch_indices:
    index = dataset.patch_indices[patch]
    fig.add_trace(go.Scatter(x=dataset.x[index,0],
                                y=dataset.x[index,1],
                                mode='markers',
                                marker=dict(
                                    color=dataset.y[case_n,index,channel],
                                    colorscale='Viridis',
                                    coloraxis='coloraxis'
                                    ),
                                text=dataset.y[case_n,index,channel],
                                name=patch)) # hover text goes here

fig.update_layout(coloraxis = {'colorscale':'Inferno', 'cmid': 0.5, 'reversescale': False}, 
                  font_family="Times New Roman", font_color="black", template="ggplot2"#, margin=dict(l=80, r=20, t=20, b=20, pad=0)
    ,autosize=True, 
    yaxis_scaleanchor="x",plot_bgcolor='rgba(0,0,0,0)',
    showlegend=False)

fig['layout']['xaxis']['visible'] = False
fig['layout']['yaxis']['visible'] = False
fig.show()

In [ ]:
# Create figure
fig = go.Figure()
n_internal = dataset.x[:,0].shape[0]
# Add traces, one for each slider step

for patch in dataset.patch_indices:
    index = dataset.patch_indices[patch]
    fig.add_trace(go.Scatter(x=dataset.x[index,0],
                                y=dataset.x[index,1],
                                mode='markers',
                                marker=dict(
                                    color=dataset.y[case_n,index,channel],
                                    colorscale='Viridis',
                                    coloraxis='coloraxis',
                                    size = 1.25
                                    ),
                                text=dataset.y[case_n,index,channel],
                                name=patch,
                                visible=True)) # hover text goes here

sub_samples = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
#sub_samples = [0.01,0.01,0.01,0.01,0.01,0.01 ]
for step in sub_samples:
    #index = np.random.randint(low=0,high=65536,size=round(65536*(step),dtype=int)
    
    index = list(np.random.default_rng().choice(n_internal, size=int(n_internal*step), replace=False))
    fig.add_trace(go.Scatter(x=dataset.x[index,0],
                                    y=dataset.x[index,1],
                                    mode='markers',
                                    marker=dict(
                                        color=dataset.y[case_n,index,channel],
                                        colorscale='Viridis',
                                        coloraxis='coloraxis',
                                        size = 1.25
                                        ),
                                    text=dataset.y[case_n,index,channel],
                                    name='Internal Field',
                                    visible=True)) # hover text goes here

# Create and add slider
steps = []

for i in range(len(fig.data)-2):
    step = dict(
        method="update",
        args=[{"visible": [True, True] + ([False] * (len(fig.data)-2))},
              {"title": "Cavity Flow Dataset"}
              ],
        label= f'{100*sub_samples[i]:.0f}%'
    )
    step["args"][0]["visible"][i+2] = True  # Toggle i'th trace to "visible"
    steps.append(step)


sliders = [dict(
    active=10,
    currentvalue={"prefix": "Subsample Amount: ",
                  "visible":False},
    pad={"t": 0},
    steps=steps
)]

fig.update_layout(coloraxis = {'colorscale':'Inferno', 'cmid': 0.5, 'reversescale': False}, 
                  font_color="black", template="ggplot2"#, margin=dict(l=80, r=20, t=20, b=20, pad=0)
    ,autosize=True, 
    yaxis_scaleanchor="x",plot_bgcolor='rgba(0,0,0,0)',
    showlegend=False,
    sliders=sliders)

fig['layout']['xaxis']['visible'] = False
fig['layout']['yaxis']['visible'] = False
fig.show()